In [1]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from scipy.stats import skew
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import itertools

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_test = pd.read_csv('train.csv', delimiter=',')
train_test_predict = pd.DataFrame(train_test['situacao'])
predict = pd.read_csv('test.csv', delimiter=',')
predict_code = predict['sequencial_candidato']

# Dados de Treino e Teste (2006/2010)

In [3]:
train_test.shape

(7622, 24)

In [4]:
train_test.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.67,0.0,...,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,nao_eleito
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.85,6625.0,...,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,nao_eleito
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,158120.80,9301.22,2250.0,...,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,eleito
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.19,0.0,...,8,8,3001.12,375.14,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,nao_eleito
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,119820.00,2496.25,0.0,...,133,120,116416.64,875.31,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,eleito


In [5]:
train_test.tail()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
7617,2010,270000000264,ROBERTO PEREIRA URBANO,TO,PSC,3,3,3370.00,1123.33,370.00,...,16,16,3363.41,210.21,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ADVOGADO,nao_eleito
7618,2010,270000000265,RONALDO DIMAS NOGUEIRA PEREIRA,TO,PR,715,706,1045323.16,1461.99,211804.58,...,1658,1275,815638.26,491.94,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ENGENHEIRO,nao_eleito
7619,2010,270000000266,SÉRGIO PERIN,TO,PTB,2,2,2600.00,1300.00,0.00,...,2,2,2600.00,1300.00,DEPUTADO FEDERAL,MASCULINO,SUPERIOR INCOMPLETO,CASADO(A),ODONTÓLOGO,nao_eleito
7620,2010,270000000267,ÍTALO MARCEL COSTA CONCEIÇÃO,TO,PR,2,2,5175.00,2587.50,3375.00,...,2,2,5175.00,2587.50,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BANCÁRIO E ECONOMIÁRIO,nao_eleito
7621,2010,270000000501,LEONAM XAVIER GOMES,TO,PPS,13,12,36852.82,2834.83,30083.67,...,47,46,36739.62,781.69,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),SERVIDOR PÚBLICO FEDERAL,nao_eleito


# Dados de Validação (2014)

In [6]:
predict.shape

(4592, 23)

In [7]:
predict.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
0,2014,10000000135,EMERSON DA SILVA SANTOS,AC,PSOL,3,3,1580.00,526.67,0.00,...,80.0,3,3,1580.00,526.67,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),"CORRETOR DE IMÓVEIS, SEGUROS, TÍTULOS E VALORES"
1,2014,10000000142,GERALDO SILVA DOS SANTOS,AC,PSOL,5,5,3180.00,636.00,0.00,...,80.0,6,5,3130.02,521.67,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),VIGILANTE
2,2014,10000000158,CARLOS CESAR CORREIA DE MESSIAS,AC,PSB,40,38,336793.13,8419.83,1923.07,...,0.0,145,139,326869.78,2254.27,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,CASADO(A),OUTROS
3,2014,10000000161,IDESIO LUIS FRANKE,AC,PT,29,29,156719.32,5404.11,39122.32,...,25000.0,136,121,241016.07,1772.18,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),AGRÔNOMO
4,2014,10000000163,LEONARDO CUNHA DE BRITO,AC,PT,160,146,737073.00,4606.71,10000.00,...,25000.0,518,354,567401.15,1095.37,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ADVOGADO


In [8]:
predict.tail()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
4587,2014,270000000304,JOENICE PEREIRA RIBEIRO,TO,PR,7,6,6482.29,926.04,6482.29,...,0.0,7,6,6334.29,904.90,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,CASADO(A),PROFESSOR DE ENSINO MÉDIO
4588,2014,270000000305,TIAGO DE PAULA ANDRINO,TO,PP,42,42,1738508.82,41393.07,54175.62,...,100000.0,646,492,1799447.74,2785.52,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),OUTROS
4589,2014,270000000312,ETEVALDO DA PAZ NONATO,TO,PSOL,3,3,1230.00,410.00,0.00,...,0.0,3,3,1230.00,410.00,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),ELETRICISTA E ASSEMELHADOS
4590,2014,270000000329,IVARDO SANTANA,TO,PSOL,2,2,900.00,450.00,0.00,...,0.0,2,2,900.00,450.00,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),ELETRICISTA E ASSEMELHADOS
4591,2014,270000000335,MARIA LUCIA SOARES VIANA,TO,PSOL,10,9,9895.00,989.50,0.00,...,0.0,26,23,9087.52,349.52,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,CASADO(A),ADMINISTRADOR


# Analisando Correlação entre Features (Quantitativas)

In [9]:
rets3 = train_test[['quantidade_doacoes','quantidade_doadores','total_receita','media_receita','recursos_de_outros_candidatos.comites','recursos_de_pessoas_fisicas','recursos_de_pessoas_juridicas','recursos_proprios','recursos_de_partido_politico','quantidade_despesas','quantidade_fornecedores','total_despesa','media_despesa']].pct_change()  
corr = rets3.corr()
corr

,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa
quantidade_doacoes,1.000000,0.972108,0.031320,0.032295,0.289401,0.020163,0.076804,0.074304,0.085518,0.414139,0.393043,0.031576,-0.002489
quantidade_doadores,0.972108,1.000000,0.039286,0.038769,0.140766,0.020596,0.076789,0.071454,0.080401,0.400233,0.392603,0.042778,-0.001883
total_receita,0.031320,0.039286,1.000000,0.677075,0.627655,-0.000054,0.243724,0.085389,0.985904,0.031363,0.034355,0.998743,0.964901
media_receita,0.032295,0.038769,0.677075,1.000000,0.169286,-0.001621,0.260496,0.055601,0.946218,0.047223,0.052388,0.672309,0.841591
recursos_de_outros_candidatos.comites,0.289401,0.140766,0.627655,0.169286,1.000000,-0.001132,0.006858,-0.001109,0.073239,0.093287,0.079737,0.557098,0.101197
recursos_de_pessoas_fisicas,0.020163,0.020596,-0.000054,-0.001621,-0.001132,1.000000,-0.001073,0.003080,0.010947,0.042258,0.027713,-0.000185,-0.002542
recursos_de_pessoas_juridicas,0.076804,0.076789,0.243724,0.260496,0.006858,-0.001073,1.000000,-0.001214,0.139974,0.233540,0.208914,0.304118,0.176873
recursos_proprios,0.074304,0.071454,0.085389,0.055601,-0.001109,0.003080,-0.001214,1.000000,-0.000811,0.104040,0.137906,0.073660,0.031051
recursos_de_partido_politico,0.085518,0.080401,0.985904,0.946218,0.073239,0.010947,0.139974,-0.000811,1.000000,0.379391,0.345642,0.985266,0.418532
quantidade_despesas,0.414139,0.400233,0.031363,0.047223,0.093287,0.042258,0.233540,0.104040,0.379391,1.000000,0.966956,0.033653,-0.001551


# Removendo Correlações, Variáveis de Identificação e Informações Não Discriminatórias

In [10]:
train_test = train_test.drop(['uf', 'partido', 'cargo', 'ocupacao' , 'sequencial_candidato', 'nome', 'grau', 'cargo', 'estado_civil', 'ocupacao', 'situacao', 'quantidade_doadores', 'quantidade_fornecedores', 'total_despesa', 'media_despesa'], axis=1)

In [11]:
predict = predict.drop(['uf', 'partido', 'cargo', 'ocupacao' , 'sequencial_candidato', 'nome', 'grau', 'cargo', 'estado_civil', 'ocupacao', 'quantidade_doadores', 'quantidade_fornecedores', 'total_despesa', 'media_despesa'], axis=1)

In [12]:
train_test.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2006,6,16600.00,2766.67,0.0,9000.0,6300.00,1300.0,0.0,14,MASCULINO
1,2006,13,22826.00,1755.85,6625.0,15000.0,1000.00,201.0,0.0,24,FEMININO
2,2006,17,158120.80,9301.22,2250.0,34150.0,62220.80,59500.0,0.0,123,FEMININO
3,2006,6,3001.12,500.19,0.0,1150.0,1101.12,750.0,0.0,8,MASCULINO
4,2006,48,119820.00,2496.25,0.0,50878.0,0.00,68942.0,0.0,133,MASCULINO


In [13]:
train_test.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
7617,2010,3,3370.00,1123.33,370.00,0.00,0.0,3000.00,0.0,16,MASCULINO
7618,2010,715,1045323.16,1461.99,211804.58,431518.58,173500.0,28500.00,200000.0,1658,MASCULINO
7619,2010,2,2600.00,1300.00,0.00,0.00,0.0,600.00,2000.0,2,MASCULINO
7620,2010,2,5175.00,2587.50,3375.00,1800.00,0.0,0.00,0.0,2,MASCULINO
7621,2010,13,36852.82,2834.83,30083.67,500.00,0.0,6269.15,0.0,47,MASCULINO


In [14]:
predict.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2014,3,1580.00,526.67,0.00,1500.00,0.00,0.00,80.0,3,MASCULINO
1,2014,5,3180.00,636.00,0.00,3100.00,0.00,0.00,80.0,6,MASCULINO
2,2014,40,336793.13,8419.83,1923.07,65700.00,154170.06,115000.00,0.0,145,MASCULINO
3,2014,29,156719.32,5404.11,39122.32,68783.84,17000.00,6813.16,25000.0,136,MASCULINO
4,2014,160,737073.00,4606.71,10000.00,347073.00,277000.00,78000.00,25000.0,518,MASCULINO


In [15]:
predict.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
4587,2014,7,6482.29,926.04,6482.29,0.0,0.0,0.0,0.0,7,FEMININO
4588,2014,42,1738508.82,41393.07,54175.62,260000.0,1311000.0,13333.2,100000.0,646,MASCULINO
4589,2014,3,1230.00,410.00,0.00,1230.0,0.0,0.0,0.0,3,MASCULINO
4590,2014,2,900.00,450.00,0.00,900.0,0.0,0.0,0.0,2,MASCULINO
4591,2014,10,9895.00,989.50,0.00,1080.0,0.0,8815.0,0.0,26,FEMININO


# Normalizando Variáveis Quantitativas

In [16]:
numeric_feats = train_test.dtypes[train_test.dtypes != "object"].index

skewed_feats = train_test[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

train_test[skewed_feats] = np.log1p(train_test[skewed_feats])

In [17]:
numeric_feats = predict.dtypes[predict.dtypes != "object"].index

skewed_feats = predict[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

predict[skewed_feats] = np.log1p(predict[skewed_feats])

In [18]:
train_test.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2006,1.945910,9.717218,7.925761,0.000000,9.105091,8.748464,7.170888,0.0,2.708050,MASCULINO
1,2006,2.639057,10.035699,7.471278,8.798757,9.615872,6.908755,5.308268,0.0,3.218876,FEMININO
2,2006,2.890372,11.971121,9.138008,7.719130,10.438547,11.038461,10.993748,0.0,4.820282,FEMININO
3,2006,1.945910,8.007074,6.216985,0.000000,7.048386,7.004991,6.621406,0.0,2.197225,MASCULINO
4,2006,3.891820,11.693754,7.822945,0.000000,10.837206,0.000000,11.141035,0.0,4.897840,MASCULINO


In [19]:
train_test.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
7617,2010,1.386294,8.122965,7.024943,5.916202,0.000000,0.000000,8.006701,0.000000,2.833213,MASCULINO
7618,2010,6.573680,13.859838,7.288238,12.263424,12.975068,12.063939,10.257694,12.206078,7.413970,MASCULINO
7619,2010,1.098612,7.863651,7.170888,0.000000,0.000000,0.000000,6.398595,7.601402,1.098612,MASCULINO
7620,2010,1.098612,8.551788,7.858834,8.124447,7.496097,0.000000,0.000000,0.000000,1.098612,MASCULINO
7621,2010,2.639057,10.514715,7.950090,10.311771,6.216606,0.000000,8.743556,0.000000,3.871201,MASCULINO


In [20]:
predict.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2014,1.386294,7.365813,6.268471,0.000000,7.313887,0.000000,0.000000,4.394449,1.386294,MASCULINO
1,2014,1.791759,8.064951,6.456770,0.000000,8.039480,0.000000,0.000000,4.394449,1.945910,MASCULINO
2,2014,3.713572,12.727227,9.038464,7.562198,11.092869,11.945818,11.652696,0.000000,4.983607,MASCULINO
3,2014,3.401197,11.962218,8.595100,10.574474,11.138739,9.741027,8.826758,10.126671,4.919981,MASCULINO
4,2014,5.081404,13.510444,8.435486,9.210440,12.757293,12.531776,11.264477,10.126671,6.251904,MASCULINO


In [21]:
predict.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
4587,2014,2.079442,8.776983,6.831997,8.776983,0.000000,0.000000,0.000000,0.000000,2.079442,FEMININO
4588,2014,3.761200,14.368539,10.630893,10.900005,12.468441,14.086302,9.498087,11.512935,6.472346,MASCULINO
4589,2014,1.386294,7.115582,6.018593,0.000000,7.115582,0.000000,0.000000,0.000000,1.386294,MASCULINO
4590,2014,1.098612,6.803505,6.111467,0.000000,6.803505,0.000000,0.000000,0.000000,1.098612,MASCULINO
4591,2014,2.397895,9.199886,6.898210,0.000000,6.985642,0.000000,9.084324,0.000000,3.295837,FEMININO


# Transformando Variáveis Categóricas

In [22]:
train_test['sexo'] = pd.Categorical(train_test['sexo'])
train_test['sexo'] = train_test.sexo.cat.codes

In [23]:
train_test_predict['situacao'] = pd.Categorical(train_test_predict['situacao'])
train_test_predict['situacao'] = train_test_predict.situacao.cat.codes

In [24]:
predict['sexo'] = pd.Categorical(predict['sexo'])
predict['sexo'] = predict.sexo.cat.codes

# Dados de Treinamento, Teste e Validação

In [25]:
data_train = train_test[:3581]
target_train = train_test_predict[:3581]

In [26]:
data_test = train_test[3581:]
target_test = train_test_predict[3581:]

In [27]:
data_predict = predict[:]

In [28]:
data_train.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2006,1.945910,9.717218,7.925761,0.000000,9.105091,8.748464,7.170888,0.0,2.708050,1
1,2006,2.639057,10.035699,7.471278,8.798757,9.615872,6.908755,5.308268,0.0,3.218876,0
2,2006,2.890372,11.971121,9.138008,7.719130,10.438547,11.038461,10.993748,0.0,4.820282,0
3,2006,1.945910,8.007074,6.216985,0.000000,7.048386,7.004991,6.621406,0.0,2.197225,1
4,2006,3.891820,11.693754,7.822945,0.000000,10.837206,0.000000,11.141035,0.0,4.897840,1


In [29]:
data_train.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
3576,2006,1.791759,7.386018,5.779045,6.656727,0.000000,0.000000,0.000000,6.728952,1.791759,1
3577,2006,1.609438,7.369160,5.984742,7.159525,0.000000,0.000000,5.707110,0.000000,1.609438,1
3578,2006,1.098612,6.723868,6.031910,0.000000,0.000000,0.000000,6.723868,0.000000,1.386294,1
3579,2006,1.945910,7.381589,5.592925,0.000000,6.857514,6.196729,5.111988,0.000000,2.639057,1
3580,2006,3.332205,11.974589,8.678916,9.780327,11.469297,10.633473,7.767294,7.243227,4.521789,0


In [30]:
data_test.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
3581,2010,3.610918,12.972221,9.388784,12.503138,10.975944,8.926358,11.463068,0.000000,4.007333,0
3582,2010,1.386294,8.852379,7.754053,0.000000,0.000000,8.610866,7.313887,0.000000,1.386294,1
3583,2010,1.386294,7.518064,6.420532,5.673323,7.313887,0.000000,3.931826,0.000000,1.386294,1
3584,2010,0.693147,6.089045,6.089045,6.089045,0.000000,0.000000,0.000000,0.000000,0.693147,0
3585,2010,2.772589,12.744614,10.036604,0.000000,8.716208,11.100451,9.296610,12.466516,4.430817,1


In [31]:
data_test.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
7617,2010,1.386294,8.122965,7.024943,5.916202,0.000000,0.000000,8.006701,0.000000,2.833213,1
7618,2010,6.573680,13.859838,7.288238,12.263424,12.975068,12.063939,10.257694,12.206078,7.413970,1
7619,2010,1.098612,7.863651,7.170888,0.000000,0.000000,0.000000,6.398595,7.601402,1.098612,1
7620,2010,1.098612,8.551788,7.858834,8.124447,7.496097,0.000000,0.000000,0.000000,1.098612,1
7621,2010,2.639057,10.514715,7.950090,10.311771,6.216606,0.000000,8.743556,0.000000,3.871201,1


In [32]:
data_predict.head()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
0,2014,1.386294,7.365813,6.268471,0.000000,7.313887,0.000000,0.000000,4.394449,1.386294,1
1,2014,1.791759,8.064951,6.456770,0.000000,8.039480,0.000000,0.000000,4.394449,1.945910,1
2,2014,3.713572,12.727227,9.038464,7.562198,11.092869,11.945818,11.652696,0.000000,4.983607,1
3,2014,3.401197,11.962218,8.595100,10.574474,11.138739,9.741027,8.826758,10.126671,4.919981,1
4,2014,5.081404,13.510444,8.435486,9.210440,12.757293,12.531776,11.264477,10.126671,6.251904,1


In [33]:
data_predict.tail()

,ano,quantidade_doacoes,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,sexo
4587,2014,2.079442,8.776983,6.831997,8.776983,0.000000,0.000000,0.000000,0.000000,2.079442,0
4588,2014,3.761200,14.368539,10.630893,10.900005,12.468441,14.086302,9.498087,11.512935,6.472346,1
4589,2014,1.386294,7.115582,6.018593,0.000000,7.115582,0.000000,0.000000,0.000000,1.386294,1
4590,2014,1.098612,6.803505,6.111467,0.000000,6.803505,0.000000,0.000000,0.000000,1.098612,1
4591,2014,2.397895,9.199886,6.898210,0.000000,6.985642,0.000000,9.084324,0.000000,3.295837,0


# Validação Cruzada

In [34]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.winter):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", 
                 color="white" if cm[i, j] < thresh else "black", fontsize=40)
    
    plt.tight_layout()
    plt.ylabel('True label', fontsize=10)
    plt.xlabel('Predicted label', fontsize=10)

    return plt

In [35]:
def confusion_val(classifier, X_train, Y_train, cv):
    y_pred = cross_val_predict(classifier, X_train, Y_train, cv=cv)
    cm = confusion_matrix(Y_train, y_pred)
    fig = plt.figure(figsize=(5, 5))
    plot = plot_confusion_matrix(cm, classes=['Eleito','Não Eleito'], normalize=False, title='Matriz de Confusão')
    plt.show()

In [36]:
def sample_confusion_val(Y_train, y_pred):
    cm = confusion_matrix(Y_train, y_pred)
    fig = plt.figure(figsize=(5, 5))
    plot = plot_confusion_matrix(cm, classes=['Eleito','Não Eleito'], normalize=False, title='Matriz de Confusão')
    plt.show()

In [37]:
def cross_val(classifier, X_train, Y_train, cv):
    acc = cross_val_score(classifier, X_train, Y_train, cv=cv)
    prec = cross_val_score(classifier, X_train, Y_train, scoring='precision', cv=cv)
    rec = cross_val_score(classifier, X_train, Y_train, scoring='recall', cv=cv)
    f1 = cross_val_score(classifier, X_train, Y_train, scoring='f1', cv=cv)
    roc_auc = cross_val_score(classifier, X_train, Y_train, scoring='roc_auc', cv=cv)
    
    print("Accuracy: %.2f%% (%.2f%%)" % (np.ma.median(acc) * 100, acc.std() * 100))
    print("Precision: %.2f%% (%.2f%%)" % (np.ma.median(prec) * 100, prec.std() * 100))
    print("Recall: %.2f%% (%.2f%%)" % (np.ma.median(rec) * 100, rec.std() * 100))
    print("F-measure: %.2f%%, (%.2f%%)" % (np.ma.median(f1) * 100,f1.std() * 100))
    print("ROC AUC: %.2f%%, (%.2f%%)" % (np.ma.median(roc_auc) * 100,roc_auc.std() * 100))


In [38]:
#cv = 10
cv = StratifiedKFold(n_splits=100, random_state=10)

# Modelo KNN

In [ ]:
params = {'n_neighbors':[2,5,10],
          'leaf_size':[10, 30, 50],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute']}

model_knn = GridSearchCV(KNeighborsClassifier(n_jobs=10), param_grid=params, n_jobs=10, cv=cv)

model_knn.fit(data_train, target_train['situacao'])

print("Melhores Parâmetros:",model_knn.best_params_) 

### Treinamento

In [ ]:
cross_val(model_knn, data_train, target_train['situacao'], cv)

In [ ]:
confusion_val(model_knn, data_train, target_train['situacao'], cv)

### Teste

In [ ]:
cross_val(model_knn, data_test, target_test['situacao'], cv)

In [ ]:
confusion_val(model_knn, data_test, target_test['situacao'], cv)

# Modelo DesicionTree

In [ ]:
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split':[0.1,2,5], 
          'min_samples_leaf':[0.1,1,5]}

model_dt = GridSearchCV(DecisionTreeClassifier(), param_grid=params, n_jobs=10, cv=cv)

model_dt.fit(data_train, target_train['situacao'])

print("Melhores Parâmetros:",model_dt.best_params_)

### Treinamento

In [ ]:
cross_val(model_dt, data_train, target_train['situacao'], cv)

In [ ]:
confusion_val(model_dt, data_train, target_train['situacao'], cv)

### Teste

In [ ]:
cross_val(model_dt, data_test, target_test['situacao'], cv)

In [ ]:
confusion_val(model_dt, data_test, target_test['situacao'], cv)

# Modelo XGBoost

In [ ]:
params = {'max_depth':[1,6,11],
          'min_child_weight':[0.1,1,5],
          'gamma':[0.1, 1, 5],
          'subsample':[0.1, 1, 5],
          'colsample_bytree':[0.1, 1, 5],
          'reg_alpha':[0.1, 1, 5]}
        
model_xg = GridSearchCV(XGBClassifier(n_jobs=10), param_grid=params, n_jobs=10, cv=cv)

model_xg.fit(data_train, target_train['situacao'])

print("Melhores Parâmetros:",model_xg.best_params_)

### Treinamento

In [ ]:
cross_val(model_xg, data_train, target_train['situacao'], cv)

In [ ]:
confusion_val(model_xg, data_train, target_train['situacao'], cv)

### Teste

In [ ]:
cross_val(model_xg, data_test, target_test['situacao'], cv)

In [ ]:
confusion_val(model_xg, data_test, target_test['situacao'], cv)

# Predição